Citybik.es

Citybik.es is a website that offers an Application Programming Interface (or API, for short) for the usage of bike-sharing services throughout the world. Among the others, data for one of Turin’s bike sharing system is available. The information available is at a “station” granularity. This means that all the data available regards the bike stations: some of the useful information available is the station name, its position (in terms of latitude and longitude), the number of available bikes and the number of free docks. **The data is offered in near real-time (i.e. it is updated every 15-30 minutes)** 


The API endpoint to request the data about for the Bike service is the following: http://api.citybik.es/v2/networks/to-bike. This dataset is in the JSON (JavaScript Object Notation) format.

**Exercise Citybik.es dataset**

Load the Citybik.es dataset as a Python dictionary. Use of the json module. Count and print the number of active stations (a station is active if its extra.status field is "online"). Count and print the total number of bikes available (field free_bikes) and the number of free docks (field empty_slots) throughout all stations. Given the coordinates (latitude, longitude) of a point (e.g. 45.074512, 7.694419), identify the closest bike station to it that has available bikes. For computing the distance among two points (given their coordinates), you can use the function distance_coords() defined in the code snippet below (which is an implementation of the great-circle distance): 
    
    
from math import cos, acos, sin 
defdistance_coords(lat1, lng1, lat2, lng2):
     """Compute the distance among two points.""" 
     deg2rad = lambda x: x * 3.141592 / 180 
     lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
     R = 6378100 # Radius of the Earth, in meters return
     R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2)) 

#### Exercise 2.1 Load the Citybik.es dataset as a Python dictionary. Use of the json module.

In [39]:
import json
with open("to-bike.json") as f:
    dataset = json.load(f)

In [40]:
dataset

{'network': {'company': ['Comunicare S.r.l.'],
  'href': '/v2/networks/to-bike',
  'id': 'to-bike',
  'location': {'city': 'Torino',
   'country': 'IT',
   'latitude': 45.07098200000001,
   'longitude': 7.685676},
  'name': '[TO]BIKE',
  'source': 'http://www.tobike.it/frmLeStazioni.aspx?ID=22',
  'stations': [{'empty_slots': 9,
    'extra': {'number': 4,
     'reviews': 340,
     'score': 3.9,
     'status': 'online',
     'uid': '253'},
    'free_bikes': 10,
    'id': '9f705b5e090de99e976f4ac6c6911571',
    'latitude': 45.072882,
    'longitude': 7.667951,
    'name': 'Porta Susa 1',
    'timestamp': '2021-05-24T07:45:17.767000Z'},
   {'empty_slots': 9,
    'extra': {'number': 5,
     'reviews': 221,
     'score': 3.4,
     'status': 'online',
     'uid': '254'},
    'free_bikes': 6,
    'id': '53a856e2cd42e9bfbc022f6cdc6d2e1c',
    'latitude': 45.072642,
    'longitude': 7.670337,
    'name': 'Cernaia',
    'timestamp': '2021-05-24T07:45:17.767000Z'},
   {'empty_slots': 15,
    'ext

The dataset dictionary now contains all the information needed.

In [41]:
dataset.keys()

dict_keys(['network'])

In [42]:
type(dataset["network"])

dict

In [43]:
dataset["network"].keys()

dict_keys(['company', 'href', 'id', 'location', 'name', 'source', 'stations'])

In [44]:
type(dataset["network"]["stations"])

list

In [49]:
dataset["network"]["stations"][4]

{'empty_slots': 2,
 'extra': {'number': 17,
  'reviews': 317,
  'score': 4.0,
  'status': 'online',
  'uid': '265'},
 'free_bikes': 12,
 'id': '7bab9c15c77104412b391ebe188346af',
 'latitude': 45.07396293061647,
 'longitude': 7.684050574676576,
 'name': 'Porta Palatina',
 'timestamp': '2021-05-24T07:45:17.767000Z'}

This contains all the relevant information we need for the next exercises.

#### Exercise 2.2 Count and print the number of active stations (a station is active if its extra.status field is "online"). 

In [47]:
active_stations = [ station for station in dataset["network"]["stations"] if station["extra"]["status"] == "online" ]
print("Number of active stations", len(active_stations))

Number of active stations 36


#### Exercise 2.3 Count and print the total number of bikes available (field free_bikes) and the number of free docks (field empty_slots) throughout all stations.

we need to count the total number of bikes available(free_bikes) and the total number of free docks(empty_slots).
This information is available at a "single station" level, so we are going to aggregate the information at a "whole system" level.

In [17]:
bikes_avail = sum([ station["free_bikes"] for station in dataset["network"]["stations"]])
free_docks = sum([ station["empty_slots"] for station in dataset["network"]["stations"]])
print("Bikes available", bikes_avail)
print("Free docks", free_docks)

Bikes available 238
Free docks 214


#### Exercise 2.4 Given the coordinates (latitude, longitude) of a point (e.g. 45.074512, 7.694419), identify the closest bike station to it that has available bikes. For computing the distance among two points (given their coordinates), you can use the function distance_coords() defined in the code snippet below (which is an implementation of the great-circle distance): 

computing distances is based on the **great-circle distance**, which approximates the Earth to a sphere, and computing the distances therein (on a sphere, the shortest path between two points is the arc of a circumference centered in the same center as the sphere, and passing by the two points).

In [50]:
from math import cos, acos, sin

def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

In [30]:
def distance_from_point(dataset, lat, lng):
    closest = (None, None) #  store the information (closest_station, closest_distance) in the closest tuple(none, none).
    for station in dataset["network"]["stations"]:
        closest_station, closest_distance = closest
        current_distance = distance_coords(lat, lng, station["latitude"], station["longitude"])
        # if closest_distance is None, then we are at the first
        # loop execution where the station has available bikes.
        # In that case, we save the current station as the 
        # closest one (as we do not have any other stations available).
        # From the next cycle on, to update `closest`, we need
        # the station to actually be closer than the already saved one.
        #Now, we can iterate over all stations and select the closest one to our point of interest (making sure that the station has bikes available, as request by the exercise).
        if station["free_bikes"] > 0 and (closest_distance is None or current_distance < closest_distance):
            closest = (station, current_distance)
    return closest

station, distance = distance_from_point(dataset, 45.074512, 7.694419)
print("Closest station:", station["name"])
print("Distance:", distance, "meters")
print("Number of available bikes:", station["free_bikes"])

Closest station: Castello 1
Distance: 605.3679984439601 meters
Number of available bikes: 14


OR

In [38]:
def distance_from_point_2(dataset, lat, lng):
    v = [ (s, distance_coords(lat, lng, s["latitude"], s["longitude"])) for s in dataset["network"]["stations"] if s["free_bikes"] > 0 ]
    return min(v, key=lambda w: w[1])

station, distance = distance_from_point_2(dataset, 45.074512, 7.694419)
print("Closest station:", station["name"])
print("Distance:", distance, "meters")
print("Number of available bikes:", station["free_bikes"])

Closest station: Castello 1
Distance: 605.3679984439601 meters
Number of available bikes: 14
